In [13]:
from jupyter_plotly_dash import JupyterDash

import plotly.express as px

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "user123"
shelter = AnimalShelter(username, password)


##########################
# Dataframes for Filtering
##########################

# All entries
df = pd.DataFrame.from_records(shelter.read({}))

# Entries matching Water Rescue criteria
df_wr = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog", "breed":
             {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
             "sex_upon_outcome": "Intact Female",
             "age_upon_outcome_in_weeks": {"$gte": 26},
             "age_upon_outcome_in_weeks": {"$lte": 156}}))

# Entries matching Mountain/Wilderness Rescue criteria
df_mwr = pd.DataFrame.from_records(shelter.read({"animal_type": "Dog", "breed":
             {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Huskey", "Rottweiler"]},
             "sex_upon_outcome": "Intact Male",
             "age_upon_outcome_in_weeks": {"$gte": 26},
             "age_upon_outcome_in_weeks": {"$lte": 156}}))

# Entries matching Disaster/Induvidual Tracking criteria
df_dit = pd.DataFrame.from_records(shelter.read({"animal_type": "Dog","breed":
             {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
             "sex_upon_outcome": "Intact Female",
             "age_upon_outcome_in_weeks": {"$gte": 20},
             "age_upon_outcome_in_weeks": {"$lte": 300}}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Load image file
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Jordan Pribanic SNHU CS-340 Dashboard'))),
    html.Center(
        html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
            href ='www.snhu.edu',
            style = {
                'height': '5%',
                'width': '5%',
                'position': 'center',
                'padding-top': 0,
                'padding-right': 0
            }
        )
    ),
    
    html.Hr(),
    
    #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc
    html.Div(
        #Radio Items to select the rescue filter options
        html.Center(dcc.RadioItems(
            id='filter-type',
            # created the labels and keys based on the Grazioso requirements
            options=[
                {'label': 'No filter', 'value': 'NONE'},
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'MWR'},
                {'label': 'Disaster Rescue/Individual Tracking', 'value': 'DIT'}
            ],
            
            value='NONE',
            labelStyle={'display': 'inline-block', 'margin': '0 10px 0 10px'}
        )
    )),
    
    html.Hr(),
    
    dt.DataTable(
        id = 'datatable-id',
        columns = [
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data = df.to_dict('records'),

        editable=False,
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
        style_as_list_view=True,

        style_table = {
            'height': '500px', 
            'overflowY': 'auto', 
            'width': 'auto', 
            'overflowX': 'auto'
        },

        style_cell = {
            'padding': '10px'
        },

        style_header = {
            'backgroundColor': '#ddd',
            'fontWeight': 'bold'
        }
    ),
        
    html.Br(),
    html.Hr(),
    
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex', 'width': 'auto'}, children=[
             html.Div(
                 id='graph-id',
                 className='col s12 m6'
             ),
             
             html.Div(
                 id='map-id',
                 className='col s12 m6'
            )
         ]
    )
])


#############################################
# Interaction Between Components / Controller
#############################################

### Update Table ###
@app.callback(
    [Output('datatable-id','data'),
     Output('datatable-id','columns')],
    [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'NONE':
        df_filtered = df
        
    elif filter_type == 'WR':
        df_filtered = df_wr
        
    elif filter_type == 'MWR':
        df_filtered = df_mwr
        
    elif filter_type == 'DIT':
        df_filtered = df_dit
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df_filtered.columns]
    data=df_filtered.to_dict('records')
        
    return (data,columns)


### Highlights selected columns ###
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    return [{'if': { 'column_id': i }, 'background_color': '#D2F3FF'} for i in selected_columns]

 
### Update Graph ###
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    
    df_graph = dff.breed.value_counts()
    df_graph.name = "breed"
    df_graph = df_graph.to_frame()
    df_graph.reset_index(inplace=True)
    
    return [
       dcc.Graph(
           style={'width': '800px', 'height': '500px'},
           figure = px.pie(df_graph, values="breed", names="index")
       )    
    ]
    
    
### Update Map ###
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows')])
def update_map(viewData, rows):
    dff = df if viewData is None else pd.DataFrame.from_dict(viewData)
    
    if len(rows) == 0:
        rows = [0]
    
    return [
        dl.Map(style={'width': '800px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[rows[0], 13], dff.iloc[rows[0], 14]], children=[
                dl.Tooltip(dff.iloc[rows[0], 4]),
                dl.Popup([
                    html.H4("Animal Name"),
                    html.P(dff.iloc[rows[0], 9] if dff.iloc[rows[0], 9] else "No Name Provided"),
                    html.H4("Breed"),
                    html.P(dff.iloc[rows[0], 4] if dff.iloc[rows[0], 4] else "No Breed Provided"),
                    html.H4("Age (years)"),
                    html.P(round(dff.iloc[rows[0], 15] / 52, 2) if dff.iloc[rows[0], 4] else "No Age Provided"),
                    html.H4("Sex"),
                    html.P(dff.iloc[rows[0], 12] if dff.iloc[rows[0], 4] else "No Sex Provided")
                ])
            ])
        ])
    ]

app